## 2).Eye_Blinking_Detection
**we’re going to detect when the Eye is blinking. We’re going to use this function later on to press the selected key on the virtual keyboard just by blinking our eyes.**

In Eye_Detection we detected the eye and also created the vertical and horizontal across the eye.But now we will detect whether the eye is blinking or not .

<img src="picture/eye_blink.png">


There we noticed that:-

**1-When the eye are opened and closed the horizontal line length line did not that much**

**2-but the vertical line length decreased as we close the eye**

so considering these two observation we may use it to detect whether eye is closed or not.

We will take then the horizontal line as the point of reference, and from this we calculate the ratio in comparison with the vertical line.
If the the ratio goes below a certain number we will consider the eye to be closed, otherwise open.

## Importing modules

In [1]:
import cv2
import numpy as np
import dlib
from math import hypot

In [2]:
# we used the detector to detect the frontal face
detector = dlib.get_frontal_face_detector()

# it will dectect the facial landwark points 
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

font = cv2.FONT_HERSHEY_PLAIN

In [3]:
#We create a function that we will need later on to detect the medium point.
#On this function we simply put the coordinates of two points and will return the medium point 
#(the points in the middle between the two points).
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

We create a function to detect the blinking ratio where we insert the eye points and the facial landmark coordinates and we will get the ratio between these two lines.

In [4]:
def get_blinking_ratio(eye_points, facial_landmarks):
    
    #to detect the left_side of a left eye
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    
    #to detect the right_side of the left eye
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    
    #to detect the mid point for the center of top in left eye
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    
    #to detect the mid point for the center of the bottom in left eye
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))
    
    #creating the horizontal line from the left_side to the right_side of the left eye
    hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
    
    #creating the vertical line from  the center_top to center_bottom
    ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)
    
    #to calculate horizontal line distance
    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    
    #to calculate vertical line distance
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    
    #to calculate ratio
    ratio = hor_line_lenght / ver_line_lenght
    
    return ratio


In [5]:
# to open webcab to capture the image
cap = cv2.VideoCapture(0)
while True:
    _,frame = cap.read()
    
    #change the color of the frame captured by webcam to grey
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # to detect faces from grey color frame
    faces = detector(gray)
    for face in faces:
        
        #to detect the landmarks of a face
        landmarks = predictor(gray, face)
        
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2

    
        if blinking_ratio>5.7:
            cv2.putText(frame, "BLINKING", (50, 150), font, 7, (255, 0, 0))

    cv2.imshow("Frame",frame)
    
    
    key = cv2.waitKey(1)
    #close the webcam when escape key is pressed
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

ShubhangiDabral13...